In [1]:
%%writefile C:\Anaconda\Lib\CAMP.py
    
import datetime as dt
import pandas as pd
import numpy as np
import math as math 
import MathsUtilities as MUte
from scipy.optimize import curve_fit

CampConstants = {
    'k':-0.17,
    'VernalisationThreshold':1.0,
    'MethalationThreshold':0.5,
}

def LARPTQ(PTQ,maxLAR,minLAR,PTQhf):
    """Calculate the Leaf Appearance Rate at a given PTQ for a genotype of given minLAR and MaxLAR as formualted by Baumont etal 2019 Journal Expt Botany, Equation 3. 
    Args:
        PTQ: PhotoThermal Quatent (mmol PAR m-2 oCd-1)
        maxLAR: Leaf Appearance Rate at PTQ = infinity (oCd-1)
        minLAR: Leaf Appearance rate at PTQ = 0 (oCd-1)
        PTQhf: PTQ half, controls the curvature of response
    Returns:
        Leaf Appearance Rate (dHS/oCd)
    """
    return minLAR+((maxLAR-minLAR)*PTQ)/ (PTQhf+PTQ)

def CalcPTQ(Tt,Pp,lightIntensity):
    """Calculate the Photothermal Quotient for a given day in controlled environment (mmol PAR m-2 oCd-1). 
    Args:
        Tt: Mean daily thermal time (oC)
        Pp: Photoperiod (oCd-1)
        LightIntensity: radiation output when lights are on (MJ/m2/s)
    Returns:
        Photothermal Quotient (mmol PAR m-2 oCd-1)
    """
    Radn = Pp * 3600 * lightIntensity * 2  # Hours * s/h * MJ/m2/s * Coeff_PAR->Total 
    Ft = 1
    if Tt < 15:
        Ft = Tt/15
    return (Radn * 2.285 * Ft)/Tt

def CalcdHS(Tt,HS,PTQ,maxLAR,minLAR,PTQhf):
    """Calculate daily increase in Haun stage
    Args:
        Tt: Thermal time increment
        HS: Current Haun Stage
        PTQ: PhotoThermalQuotient
        maxLAR: Leaf Appearance Rate at PTQ = infinity (oCd-1)
        minLAR: Leaf Appearance rate at PTQ = 0 (oCd-1)
        PTQhf: PTQ half, controls the curvature of response
    Returns:
        Daily increas in Haun Stage
    """
    BasePhyllochron = 1/LARPTQ(PTQ,maxLAR,minLAR,PTQhf)
    StageFactor = np.interp(HS,[0,7,8,12],[1.0,1.0,1.4,1.4],False)
    Phyllochron = BasePhyllochron*StageFactor
    return Tt/Phyllochron

def CalcPpResponse(Pp, Base, Max, dHS):
    """ Calculate value of a photoperiod sensitive variable between Max and Baee values
    Args:
        Pp: Photoperiod
        Base: value below 8h Pp
        Max: value above 16h Pp
        dHS: delta haun stage
    Returns:
        variable value dependent on photoperiod and adjusted for dHS
    """
    if Pp <= 8.0:
        return Base * dHS
    if (Pp > 8.0) and (Pp < 16.0):
        return (Base + (Max * (Pp-8)/(16-8))) * dHS
    if (Pp >= 16.0):
        return Max * dHS  

def CalcBaseUpRegVrn1(Tt, dHS, BaseDVrn1):
    """ Calculate upregulation of base Vrn1
    
    Args:
        Tt: Thermal time increment
        dHS: Delta Haun stage
        BaseDVrn1: coeffociennt for Base Vrn1 expression
        
    Returns:
        delta BaseVrn1 representing the additional Vrn1 expression from base expression
    """
    if Tt < 0: 
        BaseDVrn1 = 0
    return BaseDVrn1 * dHS

def CalcColdUpRegVrn1(Tt,dHS, MaxDVrn1, k):
    """ Upregulation of Vrn1 from cold.  Is additional to base vrn1
        BaseDVrn1 in seperate calculation otherwise te same as Brown etal 2013
    Args:
        Tt: Thermal time increment
        dHS: Delta Haun stage
        MaxDVrn1: coefficient for Maximum upregulation of Vrn1
        k: The exponential coefficient determining temperature response
        
    Returns:
        delta ColdVrn1 representing the additional Vrn1 expression from cold upregulation
    """
    UdVrn1 = MaxDVrn1 * np.exp(k*Tt)
    if (Tt < 20):
        return UdVrn1 * dHS
    else:
        return -5
    
def CalcpVrn2(LPpHS,IpVrn2,DpVrn2):
    """ Potential Upregulation of Vrn2 from long photoperiod.  Actual Vrn2 expression will be less than this because it is blocked by Vrn1
    Args:
        LPpHS: Long photoperiod Haun Stage
        IpVrn2: Initial potential Vrn2 at first experience of Pp > 8 (normally at emergence)
        DpVrn2: Delta of potential Vrn2 in response to accumulation of LPpHS
    Returns:
        delta ColdVrn1 representing the additional Vrn1 expression from cold upregulation
    """
    if LPpHS <1:
        InitSlope = (IpVrn2+DpVrn2)/1
        return LPpHS * InitSlope
    else:
        return IpVrn2 + LPpHS *DpVrn2    
        
def TSHS(FLN):
    """Haun stage timing of terminal spikelet 
       Inverts equation 5 from Brown etal 2013 FLN =  2.85 + 1.1*TSHS
       Note the intercept differs, was typeo on publication
    Args:
        FLN: The final leaf number
    Returns:
        Estimation of the haun stage timing of terminal spikelet
    """
    return (FLN - 2.85)/1.1

def CAMPmodel(Out, Day, Tt, Pp, Params, Consts, TtEmerge, LightIntensity):
    """ The Cereal Anthesis Molecular Phenology model.
        Based on the ideas presented in Brown etal 2014 (Annals of Botany)
        Alterations made replacing Vrn4 notion with methalation of Vrn1, introducing VrnX to account for long day vernalisation
        Vrn2 approach changed to have hihger experssion for first HS then a lower rate
        Other alterations to implement working code base
    Args:
        Out: "FLN" or None.  If "FLN" will only return estimated FLN else will return full dataframe with daily state variable values
        Day: List, 1:EndDay representing the timesteps in model run
        Tt: List of same length as Day, representing daily temperature
        Pp: List of same length as Day, representing daily photoperiod
        Params: Dict with genotype parameters fitted by CalcCultivarVrnCoeffs() 
        Consts: Dict with crop specific constants,
        TtEmerge: Thermaltime from sowing to emergence
        LightIntensity: radiation intensity (MJ/m2/s)
    """
    # Set up Data structure and initialise values
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    DF = pd.DataFrame(index = [0], columns = ['Day','Tt','Pp'])
    IsGerminated = False
    IsEmerged = False
    IsMethalating = False
    IsVernalised = False
    IsInduced = False
    IsReproductive = False
    IsAtFlagLeaf = False
    DF.loc[0,'Day'] = 0
    DF.loc[0,'Tt'] = Tt[0]
    DF.loc[0,'Pp'] = Pp[0]
    DF.loc[0,'PTQ'] = 0
    DF.loc[0,'Stage'] = ""
    DF.loc[0,'HS'] = 0.0
    DF.loc[0,'LPpHS'] = 0.0
    DF.loc[0,'AccumTt'] = 0
    DF.loc[0,'BaseVrn1'] = 0
    DF.loc[0,'ColdVrn1'] = 0
    DF.loc[0,'MethColdVrn1'] = 0
    DF.loc[0,'Vrn1'] = 0
    DF.loc[0,'Vrn2'] = 0.0
    DF.loc[0,'Vrn3'] = 0.0
    DF.loc[0,'VrnX'] = 0.0
    DF.loc[0,'VSHS'] = 0
    DF.loc[0,'FIHS'] = 0
    DF.loc[0,'TSHS'] = 0
    DF.loc[0,'FLN'] = 2.86
    DF.loc[0,'dHS'] = 0
    DF.loc[0,'dLPpHS'] = 0
    DF.loc[0,'dBaseVrn1'] = 0
    DF.loc[0,'dColdVrn1'] = 0
    DF.loc[0,'dMethColdVrn1'] = 0
    DF.loc[0,'pVrn2'] = 0
    DF.loc[0,'dVrn3'] = 0
    DF.loc[0,'dVrnX'] = 0
    d = 1
    
    # Model daily loop
    # ^^^^^^^^^^^^^^^^
    while (IsAtFlagLeaf == False) and (d < Day[-1]):
        #Sow crop on day 1
        DF.loc[d,'Day'] = d
        if (IsGerminated == False):
            DF.loc[d,'Stage'] = 'Germination'
            IsGerminated = True
            GerminationDay = True
        # Set daily environment variables
        DF.loc[d,'Tt'] = Tt[d-1]
        DF.loc[d,'Pp'] = Pp[d-1]
        #Zero set deltas
        DF.loc[d,'dLPpHS'] = 0
        DF.loc[d,'dBaseVrn1'] = 0
        DF.loc[d,'dColdVrn1'] = 0
        DF.loc[d,'dMethColdVrn1'] = 0
        DF.loc[d,'dVrn2'] = 0
        DF.loc[d,'dVrn3'] = 0
        DF.loc[d,'dVrnX'] = 0
                
        DF.loc[d,'AccumTt'] = DF.loc[d-1,'AccumTt'] + DF.loc[d,'Tt']
        
        PropnOfDay = 1.0
        if (DF.loc[d,'AccumTt'] > TtEmerge) and (IsEmerged==False):
            IsEmerged = True
            DF.loc[d,'Stage'] = 'Emergence'
            PropnOfDay = (DF.loc[d,'AccumTt'] - TtEmerge)/DF.loc[d,'Tt'] # Calculate fraction of emergence days Tt that is not used for emergence
        
    # Calculate daily Haun Stage changes
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        DF.loc[d,'PTQ'] = CalcPTQ(DF.loc[d,'Tt'],DF.loc[d,'Pp'],LightIntensity)
        if IsEmerged==True: # Crop emerged
            # Calculate delta haun stage
            DF.loc[d,'dHS'] = CalcdHS(DF.loc[d,'Tt'],DF.loc[d-1,'HS'],DF.loc[d,'PTQ'],Params.maxLAR,Params.minLAR,Params.PTQhf)
            # Calculate delta long day photoperiod haunstage
            DF.loc[d,'dLPpHS'] = DF.loc[d,'dHS'] * CalcPpResponse(DF.loc[d,'Pp'], 0, 1, 1) * PropnOfDay
        else: # Crop not yet emerged but we still need a dHS value to drive Vrn1 expression prior to emergence
            EmergDurationFactor = 1
            if (DF.loc[d,'AccumTt'] > 90):  # Calculate EmergenceDurationFactor to slow accumulation of HS if emergence is taking a long time.  This slows Vrn1 expression under slow emergence and strange responses to delayed sowing
                EmergDurationFactor = 1 * np.exp(-0.015 * (DF.loc[d,'AccumTt']-90))
            DF.loc[d,'dHS']  = DF.loc[d,'Tt']/(1/LARPTQ(1.0,Params.maxLAR,Params.minLAR,Params.PTQhf)) * EmergDurationFactor
            DF.loc[d,'dLPpHS'] = 0
       
        #increment HS
        if IsEmerged == True:
            DF.loc[d,'HS'] = DF.loc[d-1,'HS'] +  DF.loc[d,'dHS']
            DF.loc[d,'LPpHS'] = DF.loc[d-1,'LPpHS'] + DF.loc[d,'dLPpHS']
        else:
            DF.loc[d,'HS'] = 0
            DF.loc[d,'LPpHS'] = 0
    
    # Calculate Vrn gene expression
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        if (IsVernalised==False): # do vernalisation calculations if crop not yet vernalised
        # Calculate Vrn1 deltas
            DF.loc[d,'dBaseVrn1'] = CalcBaseUpRegVrn1(DF.loc[d,'Tt'],DF.loc[d,'dHS'],Params.BaseDVrn1) 
            DF.loc[d,'dColdVrn1'] = CalcColdUpRegVrn1(DF.loc[d,'Tt'],DF.loc[d,'dHS'],Params.MaxDVrn1,Consts['k'])
            DF.loc[d,'ColdVrn1'] = max(0.0,DF.loc[d-1,'ColdVrn1'] + DF.loc[d,'dColdVrn1'])
            
        # Calculate daily methalation
            if DF.loc[d,'ColdVrn1']>= Consts['MethalationThreshold']: # ColdVrn1 expressed to threshold required for methalation to occur
                IsMethalating = True
            else:
                IsMethalating = False
                
            if (IsMethalating == True):
                DF.loc[d,'dMethColdVrn1'] = min(DF.loc[d,'ColdVrn1'] - Consts['MethalationThreshold'],
                                                max(0,DF.loc[d,'dColdVrn1']))
            
        # Calcualte expression of photoperiod sensitive genes
            DF.loc[d,'dVrnX'] = 0.0
            if (IsEmerged==True):  # Photoperiod sensitive genes only express after emergence
                if DF.loc[d-1,'MethColdVrn1'] == 0.0:  # VrnX expression only occurs if no methalation of Vrn1 has occured
                    DF.loc[d,'dVrnX'] = CalcPpResponse(DF.loc[d,'Pp'], 0.0, Params.MaxDVrnX, DF.loc[d,'dHS'])
                DF.loc[d,'pVrn2'] = CalcpVrn2(DF.loc[d,'LPpHS'],Params.IpVrn2,Params.DpVrn2)
                
        # Increment todays Vrn expression values using deltas just calculated
            DF.loc[d,'VrnX'] = DF.loc[d-1,'VrnX'] + DF.loc[d,'dVrnX']
            DF.loc[d,'BaseVrn1'] = DF.loc[d-1,'BaseVrn1'] + DF.loc[d,'dBaseVrn1']
            DF.loc[d,'MethColdVrn1'] = DF.loc[d-1,'MethColdVrn1'] + DF.loc[d,'dMethColdVrn1'] 
            # Effective expression of Vrn1 is the sum of baseVrn1, MethalatedVrn1 and Vrnx expression
            DF.loc[d,'Vrn1'] = DF.loc[d-1,'Vrn1'] + DF.loc[d,'dMethColdVrn1'] \
                                                          + DF.loc[d,'dBaseVrn1'] \
                                                          + DF.loc[d,'dVrnX'] 
            # Effective Vrn2 expression is the potential expression less that which is blocked by Vrn1
            DF.loc[d,'Vrn2'] =  max(0.0,DF.loc[d,'pVrn2'] - DF.loc[d,'Vrn1'])
                                    
        # Once Vernalisation complete, carry over vern expression values from yesterday
        else:
            DF.loc[d,'BaseVrn1'] = DF.loc[d-1,'BaseVrn1']
            DF.loc[d,'ColdVrn1'] = DF.loc[d-1,'ColdVrn1'] 
            DF.loc[d,'MethColdVrn1'] = DF.loc[d-1,'MethColdVrn1'] 
            DF.loc[d,'Vrn1'] = DF.loc[d-1,'Vrn1']
            DF.loc[d,'pVrn2'] = DF.loc[d-1,'pVrn2']
            DF.loc[d,'Vrn2'] = DF.loc[d-1,'Vrn2']
            DF.loc[d,'VrnX'] = DF.loc[d-1,'VrnX']
            
    # Set Haun stage variables
    # ^^^^^^^^^^^^^^^^^^^^^^^^
        if IsVernalised == False:
            DF.loc[d,'VSHS'] = DF.loc[d,'HS']
        else:
            DF.loc[d,'VSHS'] = DF.loc[d-1,'VSHS']
        if IsInduced == False:
            DF.loc[d,'FIHS'] = DF.loc[d,'HS']
        else: 
            DF.loc[d,'FIHS'] = DF.loc[d-1,'FIHS']
        if IsReproductive == False:
            DF.loc[d,'TSHS'] = DF.loc[d,'HS']
            DF.loc[d,'FLN'] = 2.86 + 1.1 * DF.loc[d,'TSHS']
        else:
            DF.loc[d,'TSHS'] = DF.loc[d-1,'TSHS']
            DF.loc[d,'FLN'] = DF.loc[d-1,'FLN']

    # Work out if vernalisation is complete    
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        # Vernalisation saturation occurs when Vrn1 > the vernalisation threshold and Vrn2 expression is zero
        if (IsEmerged==True) and (DF.loc[d,'Vrn1'] >= Consts['VernalisationThreshold']) and (DF.loc[d,'Vrn2'] == 0) and (IsVernalised==False):
            IsVernalised  = True
            DF.loc[d,'Stage'] = 'Vern Sat'
        
    # Then work out Vrn3 expression
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        # Vrn3 expression occurs between Vernalisation saturation and Terminal spikelet
        if (IsVernalised == True) and (IsReproductive == False):
            DF.loc[d,'dVrn3'] = CalcPpResponse(DF.loc[d,'Pp'], Params.BaseDVrn3, Params.MaxDVrn3, DF.loc[d,'dHS'])
        DF.loc[d,'Vrn3'] = min(1,DF.loc[d-1,'Vrn3'] + DF.loc[d,'dVrn3'])

    # Finally determine phenological stage
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        # Then work out phase progression based on Vrn expression
        if (DF.loc[d,'Vrn3'] >= 0.3) and (IsInduced == False):
            IsInduced = True;
        if (DF.loc[d,'Vrn3'] >=  1.0) and (IsReproductive == False):
            IsReproductive = True;
            DF.loc[d,'Stage'] = 'Term Spike'
        
        #Work out if Flag leaf has appeared.
        if DF.loc[d,'HS'] >= DF.loc[d,'FLN']:
            IsAtFlagLeaf = True    
            DF.loc[d,'Stage'] = 'FlagLeaf'
    
        #Add states to dataframe
        GerminationDay = False
        DF.loc[d,'IsGerminated'] = IsGerminated
        DF.loc[d,'IsEmerged'] = IsEmerged
        DF.loc[d,'IsMethalating'] = IsMethalating
        DF.loc[d,'IsVernalised'] = IsVernalised
        DF.loc[d,'IsReproductive'] = IsReproductive
        DF.loc[d,'IsAtFlagLeaf'] = IsAtFlagLeaf
    
        # Increment day
        d +=1
        
    if (Out == 'FLN') and (IsAtFlagLeaf == True):
        return DF.iloc[-1,:]['FLN']
    else:
        return DF
    
def CalcCultivarVrnCoeffs(inputs):
    """ Calculate VrnRate paremeters for genotype from given FLN observations
    Args:
        inputs: Dict or Series containing the following.
        FLN_LV: Final Leaf Number under long day full vernalisation conditions
        FLN_LN: Final Leaf Number under long day un-vernalisation conditions
        FLN_SV: Final Leaf Number under short day full vernalisation conditions
        FLN_SN: Final Leaf Number under short day un-vernalisation conditions
        VrnTreatTemp: Vernalisation Treatment Temperature
        VrnTreatmentTtDuration: ThermalTime Duration of vernalisation treatment
        TreatmentPTQ_L: The PTQ under long Pp,
        TreatmentPTQ_S: The PTQ under short Pp,
        TtEmerge: Thermal time from sowing to emergence
        k: Vernalisation temperature response coefficient
        VernSatThreshold: The amount of Methalated Vrn1 that is required for vernilisation saturation to occur
        MethVrn1Threshold: The amount of cold Vrn1 expression required for methalation of Vrn1 to start
        maxLAR: Leaf Appearance Rate at PTQ = infinity (oCd-1)
        minLAR: Leaf Appearance rate at PTQ = 0 (oCd-1)
        PTQhf: PTQ half, controls the curvature of response
    Returns:
        Series with values assigned to each variable in CultivarParameterList 
    """
    #Set up Data Store
    data = pd.Series()
    
    # copy some params into outputs for checking
    data['maxLAR'] = inputs['maxLAR']
    data['minLAR'] = inputs['minLAR']
    data['PTQhf'] = inputs['PTQhf']
    
    # Haun stage equivelents from imbibing to Emergence, assume a PTQ of 1.0 suitable for before emergence
    data['EmergHS'] = inputs['TtEmerge']/(1/LARPTQ(1.0,inputs['maxLAR'],inputs['minLAR'],inputs['PTQhf']))
    
    # Haun stage duration of vernalisation treatment period, assume a PTQ of 1.0 suitable for before emergence
    if inputs['TreatmentTtDuration'] <= inputs['TtEmerge']: #If Vern treatment duration complete before emergence
        data['VernTreatHS_L'] = inputs['TreatmentTtDuration']/\
                                (1/LARPTQ(1.0,inputs['maxLAR'],inputs['minLAR'],inputs['PTQhf']))
        data['VernTreatHS_S'] = inputs['TreatmentTtDuration']/\
                                (1/LARPTQ(1.0,inputs['maxLAR'],inputs['minLAR'],inputs['PTQhf']))
    else: # Add the HS when treatment finished to Emergence HS
        data['EmergToEndTreatHS_L'] = (inputs['TreatmentTtDuration'] - inputs['TtEmerge'])/\
                                      (1/LARPTQ(inputs['TreatmentPTQ_L'],inputs['maxLAR'],inputs['minLAR'],inputs['PTQhf'])) 
        data['EmergToEndTreatHS_S'] = (inputs['TreatmentTtDuration'] - inputs['TtEmerge'])/\
                                      (1/LARPTQ(inputs['TreatmentPTQ_S'],inputs['maxLAR'],inputs['minLAR'],inputs['PTQhf'])) 
        data['VernTreatHS_L'] = data['EmergHS'] + data['EmergToEndTreatHS_L']
        data['VernTreatHS_S'] = data['EmergHS'] + data['EmergToEndTreatHS_S']
    
    # Store FLN data
    data['FLN_LV']             = inputs['FLN_LV']
    data['FLN_LN']             = inputs['FLN_LN']
    data['FLN_SV']             = inputs['FLN_SV']
    data['FLN_SN']             = inputs['FLN_SN']
    
    # Calculate TSHS for each envronment set from FLNData
    data['TSHS_LV']            = TSHS( data['FLN_LV'])
    data['TSHS_LN']            = TSHS( data['FLN_LN'])
    data['TSHS_SV']            = TSHS( data['FLN_SV'])
    data['TSHS_SN']            = TSHS( data['FLN_SN'])
        
    # The soonest a plant may exhibit vern saturation if vernalisation completed early
    MinVSHS = 1.1
    
    # Minimum Haun stage duration from vernalisation saturation to terminal spikelet under long day conditions (MinHSVsTs)
                                 # Assume maximum of 3, Data from Lincoln CE (CRWT153) showed varieties that harve a high TSHS hit VS ~3HS prior to TS under these conditions
    data['MinHSVsTs']          =  min(3,data['TSHS_LV'] - MinVSHS)
    
    
    # Photoperiod sensitivity (PPS)
                                 # the difference between TSHS at 8 and 16 h pp under 
                                 # full vernalisation.
    data['PPS']                =  max(0,data['TSHS_SV'] - data['TSHS_LV'])
    
    # Calculate VSHS for each environment from TSHS and photoperiod response
    data['VSHS_LV']            =  max(MinVSHS, data['TSHS_LV'] - data['MinHSVsTs'])
    data['VSHS_LN']            =  max(MinVSHS, data['TSHS_LN'] - data['MinHSVsTs']) 
    data['VSHS_SV']            =  max(MinVSHS, data['TSHS_SV'] - (data['MinHSVsTs'] + data['PPS']))
    data['VSHS_SN']            =  max(MinVSHS, data['TSHS_SN'] - (data['MinHSVsTs'] + data['PPS']))
    
    #####################################
    # Photoperiod sensitivity calculation
    #####################################
    
    # Maximum delta for Upregulation of Vrn3 (MaxDVrn3)
                                 # Occurs under long Pp conditions
                                 # Assuming Vrn3 increases from 0 - VernSatThreshold between VS to TS and this takes 
                                 # MinHSVsTs under long Pp conditions.
    data['MaxDVrn3']           = inputs['VernalisationThreshold'] / data['MinHSVsTs']
    
    ##############################
    # Base Vern rate calculations 
    ##############################
    
    # Base delta for upredulation of Vrn3 (BaseDVrn3) 
                                 # Occurs under short Pp conditions
                                 # Assuming Vrn3 infreases from 0 - VernSatThreshold from VS to TS 
                                 # and this take 3 HS plus the additional HS from short Pp delay.
    data['BaseDVrn3']          = inputs['VernalisationThreshold'] / (data['MinHSVsTs'] + data['PPS'])
    
    # Base delta for upregulation of Vrn1 (BaseDVrn1) 
                                 # Occurs under non vernalising short Pp conditions 
                                 # Assuming Vernalisation saturation occurs when Vrn1 == VernSatThreshold  
                                 # under short Pp when no Vrn2 or VrnX are expressed and without vernalisation 
                                 # so BaseDVrn1 will be the only process creating progress toward vernalisation
                                 # Need to include time from imbib to emerge in duration 
    data['BaseDVrn1']          = inputs['VernalisationThreshold'] / (data['VSHS_SN'] + data['EmergHS'])
    
    #############################################
    # Cold Vernalisation Sensitivity Calculations
    #############################################
    
    # Use SV treatment to calculate cold response as this is not confounded by Vrn2 expression
                                 # First calculated Base Vrn1 expression at VSHS
    data['BaseVrn1AtVSHS_SV']  =  (data['VSHS_SV']+data['EmergHS']) * data['BaseDVrn1']
    
    # Then calculate how much Cold Vrn1 expression is methalated at VSHS
                                 # Assumes Vrn1 additional to baseVrn1 required to get to VS is from methalated cold Vrn1
    data['MethColdVern1AtTrans_SV'] =  max(0,inputs['VernalisationThreshold'] - data['BaseVrn1AtVSHS_SV'])
    
    # ColdVrn1AtVSHS_SV is the amount of Vrn1 upregulation caused by cold that would be required to give the 
                                 # required MethVern1AtVSHS_SV
                                 # This is the amount of cold Vrn1 that was methalated plus the amount needed to reach the methalation threshold
    data['ColdVrn1AtVSHS_SV']  = inputs['MethalationThreshold'] + data['MethColdVern1AtTrans_SV']
    
    # The haun stage duration over which vernalisation temperatures will have an effect.  Will be the minimum of the cold 
    # treatment duration (VernTreatHS) and the HS when vernalisation occurs as cold exposure after this is irrelevent
    data['HSColdEffect']       = min(data['VSHS_SV']+data['EmergHS'], data['VernTreatHS_S']);
    
    # Cold induced delta upregulation of Vrn1 at treatment temperature ('DVrn1AtVrnTreatTempt')
                                 # Calculated as the amount of cold Vrn1 upregulation divided by the HS duration of Vernaliation treatment
    data['DVrn1AtVrnTreatTemp'] = data['ColdVrn1AtVSHS_SV']/data['HSColdEffect']
    
    # Vernalisation Sensitivity measured simply as the difference between V and N treatuemnts under short Pp
    data['VS']                  =  data['VSHS_SN'] - data['VSHS_SV'] 
    
    # Maximum upregulation delta Vrn1 (MaxDVrn1)
                                 # The rate of dVrn1/HS at 0oC.  Calculate by rearanging UdVrn1 equation
                                 # dVrn1 = MaxdVrn1 * np.exp(k*VrnTreatTemp) as DVrn1 At VrnTreatTemp' is known 
                                 # calcMaxDVrn sets MaxDVrn1 to vero for genotypes with VS < 0.5 as these varieties are insensitive
                                 # and calculated rates of MaxDVrn1 are simply amplifying noise
    data['MaxDVrn1']           = calcMaxDVrn1(data['DVrn1AtVrnTreatTemp'],
                                              inputs['VrnTreatTemp'],
                                              inputs['k'],
                                              data['VS'])
    
    
    ######################################
    # Photoperiod effects on vernalisation
    ######################################
    
    # Vernalisation photoperiod sensitivity, how much does exposure to long photoperiod delay (negative) or acellerate 
                                # (positive) the timing of VS
    data['VPPS']               = 1-((data['VSHS_LN'] + data['EmergHS']) * data['BaseDVrn1'])
    
    # for genotypes that have a negative VPPS we calculate Vrn2 expression parameters using Long Pp data where Vrn2 is expressed
    # With vernalisation under long Pp the amount of Vrn 2 expression expressed at VSHS will be equal to Vernalisation threshold
    data['pVrn2AtVSHS_LV']     = inputs['VernalisationThreshold']
    # Unvernalised under long Pp conditions Vrn2 experssion will be equal to the amount of BaseVrn1 expressed at VSHS
    data['pVrn2AtVSHS_LN']     = (data['VSHS_LN'] + data['EmergHS']) * data['BaseDVrn1']

    # Assume potential Vrn2 expression will be zero at emergence because need long Photoperiod to upregulate
                                # Fit linear regression to pVrn2 expression at VSHS for V and N conditions to define potential
                                # Vrn2 expression under long day conditions
    Vrn2calc                    = calcVrn2Coeffs(data['VPPS'],
                                                data['BaseDVrn1'],
                                                data['EmergHS'],
                                                data['pVrn2AtVSHS_LV'],
                                                data['pVrn2AtVSHS_LN'],
                                                data['VSHS_LV'],
                                                data['VSHS_LN'])
    data['IpVrn2']  = Vrn2calc[0]
    data['DpVrn2']   = Vrn2calc[1]
    
     # Maximum delta upregulation for VrnX (MaxDVrnX)
                                 # This is an unidentified gene that accellerates vernalisation under long day conditions.
                                 # Only relevent for varieties that have MaxDVrn2 less than 1 (i.e no short day vernalisation response from Vrn2)
                                 # Estimate as the rate of additional Vrn1 expression needed to get to VSHS_LN over and above BaseVrn1 expression a this point
    data['BaseVrn1AtVSHS_LN']   = min(inputs['VernalisationThreshold'],(data['VSHS_LN'] + data['EmergHS']) * data['BaseDVrn1'])
    data['MaxDVrnX']            = (inputs['VernalisationThreshold']-data['BaseVrn1AtVSHS_LN'])/(data['VSHS_LN'])
    
    return data     

""" Calculate MaxDVrn1 from DVrn1 estimated at known vernalisation temperature
Args:
    DVrn1: Upregulation of Vrn1 calculated at vernalisation temperature
    Vt: Vernalisation temperature
    k: Exponential coefficient
    p: Exponential asymtote as a proportion of MaxDVrn1
Returns:
    An estimate of MaxDVrn1
"""
def calcMaxDVrn1(DVrn1AtVrnTreatTemp,VrnTreatTemp,k,VS):
    if VS > 0.5:
        return DVrn1AtVrnTreatTemp/ np.exp(k * VrnTreatTemp)
    else:
        return 0.0
                                       

""" Calculate Vrn2 rate parameters
Args:
    VPPS: Vernalisation Photo Period Sensitivity
    pVrn2AtVSHS_LV: The potential Vrn2 expressoin at the time of Vern saturation under LV conditions
    pVrn2AtVSHS_LN: The potential Vrn2 expressoin at the time of Vern saturation under LN conditions
    VSHS_LV: Vernalisation Saturation Haun Stage under LV conditions
    VSHS_LN: Vernalisation Saturation Haun Stage under LN conditions
Returns:
    tupple (a,b) where a is the initial pVrn2 expressed on day of emergence and b is the increase in pVrn2/HS as the plant develops
"""
def calcVrn2Coeffs(VPPS,BaseDVrn1,EmergHS,pVrn2AtVSHS_LV,pVrn2AtVSHS_LN,VSHS_LV,VSHS_LN):
    a = 0
    b = 0
    if (VPPS < 0) and (BaseDVrn1 < 0.4):
        Ff = 1
        b = (pVrn2AtVSHS_LN-pVrn2AtVSHS_LV)/(VSHS_LN-VSHS_LV)
        if (b >= (BaseDVrn1*0.99)): 
            Ff = 1.001
            b = (pVrn2AtVSHS_LN-(pVrn2AtVSHS_LV*Ff))/(VSHS_LN-VSHS_LV)
        a = (pVrn2AtVSHS_LV*Ff)-(VSHS_LV*b)
    return (a,b)



Overwriting C:\Anaconda\Lib\CAMP.py
